In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0 : int(n * 0.7)]
val_df = df[int(n * 0.7) : int(n * 0.9)]
test_df = df[int(n * 0.9) :]

num_features = df.shape[1]


Normalization of data


In [ ]:
# train_mean = train_df.mean()
# train_std = train_df.std()

# train_df = (train_df - train_mean) / train_std
# val_df = (val_df - train_mean) / train_std
# test_df = (test_df - train_mean) / train_std


In [ ]:
import sys

sys.path.append("../")

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from machine_learning.dataset_gen.station_dataframe_generator import StationDataframeGenerator

mpl.rcParams["figure.figsize"] = (8, 6)
mpl.rcParams["axes.grid"] = False


Download example weather data


In [ ]:
pm10_data_station_814 = StationDataframeGenerator.generate_station_dataframe(814, range(2019, 2022))
# Beispiel DataFrame
df = pm10_data_station_814.copy()

# Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten interpolieren
max_gap_size = pd.Timedelta(hours=5)

# Finde die Indizes der NaN-Werte
nan_indices = df[df["pm10"].isnull()].index

# Finde die Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten
gap_start_indices = []
current_gap_start = None
last_index = None

for index in nan_indices:
    if current_gap_start is None:
        current_gap_start = index
        last_index = index
    elif index - last_index > pd.Timedelta(hours=1):
        if last_index - current_gap_start < max_gap_size:
            gap_start_indices.append([current_gap_start, last_index])
            current_gap_start = index
        else:
            current_gap_start = index

    last_index = index


for start_index, end_index in gap_start_indices:
    length = end_index - start_index
    if length < pd.Timedelta(hours=2):
        length = pd.Timedelta(hours=2)
    real_start = start_index - length
    real_end = end_index + length
    df.loc[real_start:real_end, "pm10"] = df.loc[real_start:real_end, "pm10"].interpolate(method="time")


# Interpoliere die Lücken mit bis zu 5 aufeinanderfolgenden NaN-Werten
for start_index, end_index in gap_start_indices:
    length = end_index - start_index
    if length < pd.Timedelta(hours=2):
        length = pd.Timedelta(hours=2)
    real_start = start_index - length
    real_end = end_index + length
    df.loc[real_start:real_end, "pm10"] = df.loc[real_start:real_end, "pm10"].interpolate(method="time")

df_dropped = df.dropna()
df_dropped


Feature Engineering

- Time to Sinus/Cosinus


In [ ]:
df = pd.read_feather("../../data/pollution/processed/interpolated/pm10/814.feather")
df.set_index("timestamp")

In [ ]:
df.describe().transpose()


In [ ]:
timestamp_s = df_dropped.index.map(pd.Timestamp.timestamp)

day = 24 * 60 * 60
year = (365.2425) * day

df_dropped["Day sin"] = np.sin(timestamp_s * (2 * np.pi / day))
df_dropped["Day cos"] = np.cos(timestamp_s * (2 * np.pi / day))
df_dropped["Year sin"] = np.sin(timestamp_s * (2 * np.pi / year))
df_dropped["Year cos"] = np.cos(timestamp_s * (2 * np.pi / year))
df_dropped.reset_index(inplace=True)
# df_dropped = df_dropped.drop('time', axis=1)

df_dropped


In [ ]:
df = df_dropped.copy()
df


- convert wind to vector


In [ ]:
# Convert to radians.
wd_rad = df.pop("winddirection_10m") * np.pi / 180
df["winddirection_10m"] = wd_rad
df


Split data into train, validation and test


In [ ]:
# df_std = (df - train_mean) / train_std
# df_std = df_std.melt(var_name='Column', value_name='Normalized')
# plt.figure(figsize=(12, 6))
# ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
# _ = ax.set_xticklabels(df.keys(), rotation=90)


Data windowing


In [ ]:
class WindowGenerator:
    def __init__(self, input_width, label_width, shift, df, label_columns=None):
        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)}
        self.column_indices = {name: i for i, name in enumerate(df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    def __repr__(self):
        return "\n".join(
            [
                f"Total window size: {self.total_window_size}",
                f"Input indices: {self.input_indices}",
                f"Label indices: {self.label_indices}",
                f"Label column name(s): {self.label_columns}",
            ]
        )


In [ ]:
windowGenerator = WindowGenerator(input_width=24 * 7, label_width=24 * 7, shift=24 * 7, df=pd.DataFrame(windows[1]), label_columns=["pm10"])


In [ ]:
def split_window(self, features):
    inputs = features[:, self.input_slice, :]
    labels = features[:, self.labels_slice, :]
    # if self.label_columns is not None:
    #     labels = tf.stack([labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels


WindowGenerator.split_window = split_window


Define plot function


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


def plot(self: WindowGenerator, model=None, plot_col="pm10", max_subplots=3):
    inputs, labels = self.example
    fig = make_subplots(rows=3, cols=1, row_heights=[400,400,400])
    #plt.figure(figsize=(12, 8))
    plot_col_index = self.column_indices[plot_col]
    #plot_col_index = 4
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        #plt.subplot(max_n, 1, n + 1)
        #plt.ylabel(f"{plot_col} [normed]")
        fig.add_trace(
            go.Scatter(x=self.input_indices, y=inputs[n, :, plot_col_index].numpy(), name="Inputs", marker=dict(color="blue")),
            row=n+1, col=1
        )
        #plt.plot(self.input_indices, inputs[n, :, plot_col_index], label="Inputs", marker=".", zorder=-10)

        if self.label_columns:
            label_col_index = self.column_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        #label_col_index = 4

        if label_col_index is None:
            continue

        #plt.scatter(
        #    self.label_indices, labels[n, :, label_col_index], edgecolors="k", label="Labels", c="#2ca02c", s=64
        #)
        fig.add_trace(
            go.Scatter(x=self.label_indices, y=labels[n, :, label_col_index].numpy(), name="Labels", marker=dict(color="#2ca02c"),  mode="markers"),
            row=n+1, col=1
        )
        if model is not None:
            predictions = model(inputs)
            plt.scatter(
                self.label_indices,
                predictions[n, :, label_col_index],
                marker="X",
                edgecolors="k",
                label="Predictions",
                c="#ff7f0e",
                s=64,
            )
    fig.update_layout(height=1200)
    fig.show()
    #plt.xlabel("Time [h]")


WindowGenerator.plot = plot


Create Windows


In [18]:
window_size = 14 * 24  # in hours

# find consecutive data (24*14 hours)
windows = []

for index in range(df["pm10"].size - window_size):
    window_begin = df.iloc[index]
    window_end = df.iloc[index + window_size]
    if window_end["timestamp"] - window_begin["timestamp"] == pd.Timedelta(hours=window_size):
        df_copy = df[index : index + (window_size)].copy().drop("timestamp", axis=1)
        windows.append(df_copy)


In [21]:
pd.DataFrame(windows[1])
# len(windows)
# w1.split_window(df)


,temperature_2m,relativehumidity_2m,windspeed_10m,precipitation,pm10,Day sin,Day cos,Year sin,Year cos,winddirection_10m
1,0.6,82,14.8,0.0,59.5450,5.000000e-01,8.660254e-01,0.003455,0.999994,4.485496
2,0.3,83,15.6,0.0,55.2941,7.071068e-01,7.071068e-01,0.004172,0.999991,4.380776
3,0.3,85,15.2,0.0,39.3113,8.660254e-01,5.000000e-01,0.004888,0.999988,4.328417
4,0.0,87,15.9,0.0,35.3945,9.659258e-01,2.588190e-01,0.005605,0.999984,4.136430
5,-0.1,86,17.2,0.0,32.4548,1.000000e+00,7.593439e-12,0.006322,0.999980,4.066617
...,...,...,...,...,...,...,...,...,...,...
332,-2.8,72,25.8,0.0,22.1061,-7.071068e-01,7.071068e-01,0.238392,0.971169,5.288348
333,-3.2,72,24.7,0.0,26.7922,-5.000000e-01,8.660254e-01,0.239088,0.970998,5.218534
334,-3.6,72,23.9,0.0,23.7584,-2.588190e-01,9.659258e-01,0.239784,0.970826,5.113815
335,-3.9,72,21.7,0.0,21.9070,-3.058215e-12,1.000000e+00,0.240480,0.970654,4.974188


In [19]:
window_stack = tf.stack(windows)

windows_shuffled = tf.random.shuffle(window_stack, seed=42)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-05-30 12:37:52.803908: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-30 12:37:52.804997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [23]:
type(windows_shuffled)
stack = tf.stack(windows_shuffled)

stack.shape


TensorShape([24661, 336, 10])

In [24]:
type(windows_shuffled)


tensorflow.python.framework.ops.EagerTensor

Split windows into train, validation and test


In [20]:
windows_train = windows_shuffled[: int(len(windows_shuffled) * 0.7)]
windows_val = windows_shuffled[int(len(windows_shuffled) * 0.7) : int(len(windows_shuffled) * 0.9)]
widows_test = windows_shuffled[int(len(windows_shuffled) * 0.9) : int(len(windows_shuffled))]


In [72]:
data = np.array(widows_test, dtype=np.float32)

In [74]:
data.shape

(2467, 336, 10)

In [32]:
windows_train[0].shape


TensorShape([336, 10])

In [65]:
windowGenerator.label_columns_indices

{'pm10': 0}

In [26]:
windowGenerator.split_window(windows_train)


ValueError: Tensor's shape (17262, 0, 10) is not compatible with supplied shape [None, 1, None].

In [48]:
windows_train[:, :168, :].shape


TensorShape([17262, 168, 10])

In [28]:
windowGenerator.example[0][0, :, 4].numpy()

array([22.0742 , 20.804  , 15.4557 , 15.5559 , 14.777  , 17.9649 ,
       24.1646 , 25.3648 , 26.4219 , 24.1962 , 20.7458 , 15.7968 ,
       11.9078 , 12.2286 , 15.2906 , 15.0409 , 12.2815 , 12.9051 ,
       18.1371 , 21.4819 , 32.395  , 31.6812 , 27.6536 , 26.6803 ,
       24.273  , 25.9561 , 30.9413 , 29.2395 , 27.1053 , 26.7161 ,
       27.1632 , 31.5473 , 34.1147 , 34.8991 , 34.6654 , 36.1592 ,
       39.5747 , 36.0774 , 35.9749 , 39.1947 , 42.2104 , 49.467  ,
       52.0407 , 46.711  , 44.0832 , 28.5537 , 19.3784 , 18.9884 ,
       15.4813 , 11.2417 ,  9.39835,  8.98347, 14.4874 , 18.651  ,
       21.261  , 18.3737 , 18.4427 , 17.7367 , 18.21   , 20.4558 ,
       23.042  , 20.2191 , 18.334  , 16.8455 , 15.9349 , 16.4163 ,
       16.2365 , 16.6897 , 17.5128 , 21.9781 , 29.801  , 29.332  ,
       23.9799 , 22.8958 , 20.5399 , 16.8016 , 19.4504 , 24.014  ,
       29.3041 , 31.2251 , 31.8042 , 30.9156 , 28.9869 , 25.7359 ,
       24.6705 , 21.5431 , 20.1255 , 21.5482 , 22.9127 , 21.57

In [67]:
windowGenerator.example = windows_train[:, :168, :], windows_train[:, 168:, :]
windowGenerator.plot()


In [79]:
windowGenerator.example[1][0, :, 4]


<tf.Tensor: shape=(168,), dtype=float64, numpy=
array([18.6355, 20.2639, 19.4751, 26.1629, 24.5132, 20.1915, 25.3533,
       23.8761, 19.4648, 18.556 , 28.0799, 16.2431, 16.8292, 15.2089,
       14.7892, 16.213 , 16.563 , 14.9455, 14.3712, 16.2615, 17.7509,
       24.4752, 27.2149, 24.5478, 19.9224, 18.5587, 18.2432, 21.8649,
       22.4028, 31.5848, 29.3799, 23.772 , 24.8555, 26.6705, 25.9236,
       27.0877, 25.3442, 23.3367, 24.2145, 18.4307, 24.9058, 19.2805,
       18.0813, 19.8272, 19.6014, 19.4558, 21.0851, 22.9488, 22.6784,
       28.3437, 27.5771, 24.1263, 23.1071, 20.5864, 19.4405, 16.3553,
       15.7336, 17.528 , 19.0267, 17.523 , 17.0036, 17.7436, 17.1348,
       16.381 , 16.3455, 15.091 , 14.372 , 15.8322, 35.143 , 36.463 ,
       30.6534, 27.613 , 24.5119, 23.0897, 19.7559, 17.954 , 17.6168,
       18.1835, 20.0944, 18.4225, 15.3539, 13.9831, 20.1817, 18.835 ,
       18.7097, 16.2817, 17.2944, 20.4355, 16.5748, 16.7156, 16.0445,
       16.5648, 15.1894, 13.61  , 13.7222,

Database erstellen


Multi-step model


In [29]:
## 24 Stunden Vorhersage mit 24 Stunden Historie
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24, label_width=OUT_STEPS, shift=OUT_STEPS)
multi_window.example = example_inputs, example_labels
multi_window.plot()
multi_window


NameError: name 'example_inputs' is not defined

Definition of the model (Simple single shot network)


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(32, return_sequences=False))
model.add(tf.keras.layers.Dense(OUT_STEPS * num_features, kernel_initializer=tf.initializers.zeros()))
model.add(tf.keras.layers.Reshape([OUT_STEPS, num_features]))


In [ ]:
CONV_WIDTH = 3
OUT_STEPS = 24 * 14  # 14 days prediction
num_features = 1  # TODO: ??? num_features is 11 but must be one for our prediction

cnn_lstm_model = tf.keras.models.Sequential()
# Shape [batch, time, features] => [batch, CONV_WIDTH, features]
cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]))
# Shape => [batch, 1, conv_units]
cnn_lstm_model.add(tf.keras.layers.Conv1D(256, activation="relu", kernel_size=(CONV_WIDTH)))
cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
cnn_lstm_model.add(tf.keras.layers.Conv1D(512, activation="relu", kernel_size=(CONV_WIDTH)))
cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
cnn_lstm_model.add(tf.keras.layers.Conv1D(512, activation="relu", kernel_size=(CONV_WIDTH)))

cnn_lstm_model.add(tf.keras.layers.Flatten())
# LSTM
model.add(tf.keras.layers.LSTM(32, return_sequences=False))
model.add(tf.keras.layers.LSTM(32, return_sequences=False))

cnn_lstm_model.add(tf.keras.layers.Dense(512))
cnn_lstm_model.add(tf.keras.layers.Dropout(0.2))
cnn_lstm_model.add(tf.keras.layers.Dense(OUT_STEPS * num_features, kernel_initializer=tf.initializers.zeros()))
# Shape => [batch, out_steps, features] -> for each prediction step one neuron
cnn_lstm_model.add(tf.keras.layers.Reshape([OUT_STEPS, num_features]))


Compile Method


In [31]:
MAX_EPOCHS = 50
PATIENCE = 10  # for early-stopping; number of epochs where the val_loss must decrease -> otherwise training is stopped

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")  # for tensorboard


def compile_fit(model, window, patience=PATIENCE):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=patience, mode="min")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(), optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )

    return model.fit(
        window.train, epochs=MAX_EPOCHS, validation_data=window.val, callbacks=[early_stopping, tensorboard_callback]
    )
